In [1]:
!pip install keras
!pip install sklearn
!pip install matplotlib
!pip install -U -q PyDrive

Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple
Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple
  Running setup.py bdist_wheel for sklearn ... - done
  Stored in directory: /content/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file_import = drive.CreateFile({'id':'1AfKIkqPd3J9RUUuPNUtuj6E6E6Vh9QH9'})
file_import.GetContentFile('fast_text_embeddings.npy') 
file_import = drive.CreateFile({'id':'1IBd-lqhmNqKz-G3n5naOpflq4aLAOo9z'})
file_import.GetContentFile('sample_submission.csv') 

In [0]:
file_import = drive.CreateFile({'id':'14BQIrJoMr15-7e1Ijn4JIRs-RP2oSBJh'})
file_import.GetContentFile('y_train_full.npy') 
file_import = drive.CreateFile({'id':'1TmZd86YdR3_UqGLD-C2dJlca4gSHuE7m'})
file_import.GetContentFile('y_train.npy') 
file_import = drive.CreateFile({'id':'1JhZvB2sOfcZhNcMFG6JvuJSZawHjiyAV'})
file_import.GetContentFile('y_val.npy') 
file_import = drive.CreateFile({'id':'13pc10T15c2_fDnE2HqtAlR9FN2A5PDBD'})
file_import.GetContentFile('y_test.npy') 

In [0]:
file_import = drive.CreateFile({'id':'1pNFVlUX1DP_RAJVOuoC6Ep3fpZ0CV-hL'})
file_import.GetContentFile('X_train_full.npy') 
file_import = drive.CreateFile({'id':'12d0Qjxrf3xQRyU3umap47Sg3neYPaa0f'})
file_import.GetContentFile('X_train.npy') 
file_import = drive.CreateFile({'id':'1q0_wlpIl629IeUfH2VqiQ7UD3zAOOAs-'})
file_import.GetContentFile('X_val.npy') 
file_import = drive.CreateFile({'id':'1a3weNKFr7XMVGOTlaen33ropvFyj4jN6'})
file_import.GetContentFile('X_test.npy') 
file_import = drive.CreateFile({'id':'1sYvkToH21T8U8dIyO0TBhPeaKr_OkopS'})
file_import.GetContentFile('X_submission.npy')

In [0]:
file_import = drive.CreateFile({'id':'1FOfcAROe481NiYkkezqlGJfadYvnPZqF'})
file_import.GetContentFile('plot_history.py') 
file_import = drive.CreateFile({'id':'1YsDs9rt7I7bSc7UTNFZfoQZcsG66ND_r'})
file_import.GetContentFile('roc_auc_callback.py')

In [7]:
import numpy as np
import os

from plot_history import plot_history
from roc_auc_callback import RocAucCallback

from keras.models import Model
from keras.layers.merge import concatenate, multiply
from keras.layers import *
from keras.models import Model
from keras.initializers import Constant
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, LambdaCallback, Callback, ReduceLROnPlateau

Using TensorFlow backend.


In [0]:
from sklearn.metrics import roc_auc_score
currentEpoch = 0

class roc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return
      
    def on_epoch_end(self, epoch, logs={}):
        global currentEpoch
        currentEpoch = currentEpoch + 1
        if(currentEpoch % 2 != 0):      
            return
          
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [9]:
X_train_full = np.load('X_train_full.npy')
X_submission = np.load('X_submission.npy')
X_train = np.load('X_train.npy')
X_val = np.load('X_val.npy')
X_test = np.load('X_test.npy')

y_train_full = np.load('y_train_full.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')
y_test = np.load('y_test.npy')

print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(102124, 400) (25532, 400) (31915, 400)
(102124, 6) (25532, 6) (31915, 6)


In [0]:
from keras.layers import *
from keras.layers.merge import *
from keras.models import Sequential
import keras.initializers as initializers

class SelfAttention(Layer):
    def __init__(self, activation = 'tanh', kernel_initializer = 'glorot_uniform', **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.activation = activation
        self.kernel_initializer = initializers.get(kernel_initializer)
        
    def build(self, input_shape):
        time_steps = input_shape[1]
        dimensions = input_shape[2]
        
        self.attention = Sequential(name = 'self_attention')
        #attention matrix
        self.attention.add(Dense(
            dimensions, 
            activation = self.activation, 
            kernel_initializer = self.kernel_initializer,
            input_shape=(time_steps, dimensions,)
        ))
        #attention vector
        self.attention.add(Dense(1, activation = self.activation, kernel_initializer = self.kernel_initializer))
        self.attention.add(Flatten())
        #attention softmax
        self.attention.add(Activation('softmax'))
        #attention matrix
        self.attention.add(RepeatVector(dimensions))
        self.attention.add(Permute([2,1]))
        
        self.trainable_weights = self.attention.trainable_weights
        self.non_trainable_weights= self.attention.non_trainable_weights
        self.built = True
        
    def call(self, inputs):
        attention = self.attention(inputs)
        return Multiply()([inputs, attention])
    
    def compute_output_shape(self, input_shape):
        return input_shape
    

In [0]:
def conv_bn(inputLayer, filtersNumber, filterSize, strides, padding):
    conv = Conv1D(filters = filtersNumber, kernel_size = filterSize, strides = strides, padding = padding, kernel_initializer = 'he_uniform')(inputLayer)
    return BatchNormalization()(conv)

In [0]:
def build_extractor(features, dropout = 0.2):
  extractor = concatenate([MaxPooling1D()(features), AveragePooling1D()(features)])
  extractor = conv_bn(extractor, 92, 2, 2, 'valid')
  extractor = SpatialDropout1D(dropout)(extractor)
  extractor = concatenate([MaxPooling1D()(features), AveragePooling1D()(features)])
  extractor = conv_bn(extractor, 128, 2, 2, 'valid')
  extractor = SpatialDropout1D(dropout)(extractor)
  extractor = concatenate([GlobalMaxPooling1D()(features), GlobalAveragePooling1D()(features)])

  return extractor

In [13]:
maxWords = 30000
maxSequenceLengthInWords = 400
embeddingDimension = 300
filtersNumber = 64
reductionFiltersNumber = 128

input_layer = Input(shape=(maxSequenceLengthInWords,))

embedding_main_layer = Embedding(
    maxWords,
    output_dim=embeddingDimension,
    input_length=maxSequenceLengthInWords,
    weights=[np.load('fast_text_embeddings.npy')],
    trainable=False
)(input_layer)
embedding_main_layer = SpatialDropout1D(0.2)(embedding_main_layer)

embedding_learnable_layer = Embedding(
    maxWords,
    output_dim=embeddingDimension,
    input_length=maxSequenceLengthInWords,
    trainable=True
)(input_layer)
embedding_learnable_layer = SpatialDropout1D(0.2)(embedding_learnable_layer)

embedding_layer = average([embedding_main_layer, embedding_learnable_layer])

recurrent = Bidirectional(GRU(150, dropout = 0.2, recurrent_dropout = 0.2, return_sequences = True))(embedding_layer)
recurrent = SelfAttention()(recurrent)
recurrent = SpatialDropout1D(0.2)(recurrent)

features = build_extractor(recurrent)

output_layer = Dense(256, activation = 'elu')(features)
output_layer = BatchNormalization()(output_layer)
output_layer = Dropout(0.5)(output_layer)
output_layer = Dense(256, activation = 'elu')(output_layer)
output_layer = BatchNormalization()(output_layer)
output_layer = Dropout(0.5)(output_layer)
output_layer = Dense(6, activation = 'sigmoid')(output_layer)

model = Model(inputs=[input_layer], outputs=[output_layer])

model.summary()

model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 400, 300)     9000000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 400, 300)     9000000     input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 400, 300)     0           embedding_1[0][0]                
__________________________

In [0]:
import math
from keras.callbacks import LearningRateScheduler

initialLr = 0.001
dropRate = 0.15

lrRates = []

def step_decay(epoch):
  lr = initialLr * math.pow(2, -dropRate * epoch)
  lrRates.append(lr)
  return lr

lrScheduler = LearningRateScheduler(step_decay)

In [0]:
earlyStopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=10,
    verbose=0, 
    mode='auto'
)
rocAuc = roc_callback(training_data=(X_train, y_train), validation_data=(X_val, y_val))
reduceLr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, cooldown = 10, mode='auto')

history = model.fit(
    X_train, 
    y_train,
    batch_size=32, 
    epochs=32, 
    validation_data=(X_val, y_val), 
    callbacks=[lrScheduler, reduceLr, earlyStopping, rocAuc]
)

Train on 102124 samples, validate on 25532 samples
Epoch 1/32
102124/102124 [==============================] - 6105s 60ms/step - loss: 0.0811 - acc: 0.9740 - val_loss: 0.2172 - val_acc: 0.9146===>.] - ETA: 0s - loss: 0.0811 - acc: 0.974
Epoch 2/32
 10016/102124 [=>............................] - ETA: 1:26:21 - loss: 0.0550 - acc: 0.9805102112/102124 [============================>.] - ETA: 0s - loss: 0.0530 - acc: 0.9810102124/102124 [==============================] - 6110s 60ms/step - loss: 0.0530 - acc: 0.9810 - val_loss: 0.0587 - val_acc: 0.9798
roc-auc: 0.9915 - roc-auc_val: 0.9856                                                                                                    
Epoch 3/32
102124/102124 [==============================] - 6227s 61ms/step - loss: 0.0471 - acc: 0.9825 - val_loss: 0.0915 - val_acc: 0.9677===>.] - ETA: 0s - loss: 0.0471 - acc: 0.982
Epoch 4/32
 10016/102124 [=>............................] - ETA: 1:28:00 - loss: 0.0409 - acc: 0.9844102112/102124 [======

In [0]:
plot_history(history)